In [1]:
import os
import glob
import numpy as np
import pandas as pd

from multiprocessing import Pool

import snpy
print(f'SNooPy version: v{snpy.__version__}')

DATA_DIR = 'data'
OUT_DIR = 'fits'
SNOOPY_MODEL = 'max_model'
PROCESSES = 8

SNooPy version: v2.5.3


In [2]:
def get_parameter(sn, parameter, include_sys=False):
    """Obtains the value of the given parameter and
    the total (systematics + statistical) uncertainty
    of a fitted SN.
    
    Parameters
    ==========
    sn: SNooPy object
        SN in a SNooPy object.
    parameter: str
        Parameter to extract.
    include_sys: bool
        If `True`, systematic uncertainties are 
        included.
        
    Returns
    =======
    value: float
        Value of the parameter.
    err: float
        Total uncertainty in the parameter. 
    """
    
    value = sn.parameters[parameter]
    stats_err = sn.errors[parameter]
    if include_sys:
        sys_err = sn.systematics()[parameter]
    else:
        sys_err = 0.0
    err = np.sqrt(stats_err**2 + sys_err**2)
    
    return value, err

In [3]:
def extract_sn_params(sn_file):
    """Fits and extract the light-curve parameters
    from a SN.
    
    Parameters
    ----------
    sn_file: str
        Light-curve file in SNooPy format.
        
    Returns
    -------
    params_dict: dictionary
        Dictionary with the light-curve parameters,
        name and redshift
    """
    sn = snpy.import_lc(sn_file)
    sn.choose_model(SNOOPY_MODEL)
    try:
        sn.fit()

        outfile = os.path.join(OUT_DIR,
                              f'{sn.name}.snpy')
        sn.save(outfile)

        Tmax, Tmax_err = get_parameter(sn, 'Tmax')
        st, st_err = get_parameter(sn, 'st')
        if 'J' in sn.allbands():
            Jmax, Jmax_err = get_parameter(sn, 'Jmax')
        else:
            Jmax = Jmax_err = np.nan
        if 'H' in sn.allbands():
            Hmax, Hmax_err = get_parameter(sn, 'Hmax')
        else:
            Hmax = Hmax_err = np.nan

        params_dict = {'name':sn.name,
                       'z':sn.z,
                       'Tmax':Tmax,
                       'Tmax_err':Tmax_err,
                       'st':st,
                       'st_err':st_err,
                       'Jmax':Jmax,
                       'Jmax_err':Jmax_err,
                       'Hmax':Hmax,
                       'Hmax_err':Hmax_err,}
    except:
        params_dict = {'name':sn.name,
                       'z':sn.z,
                       'Tmax':np.nan,
                       'Tmax_err':np.nan,
                       'st':np.nan,
                       'st_err':np.nan,
                       'Jmax':np.nan,
                       'Jmax_err':np.nan,
                       'Hmax':np.nan,
                       'Hmax_err':np.nan,}

    return params_dict

In [4]:
sn_files = [file for file in glob.glob(f'{DATA_DIR}/*')]
results = Pool(PROCESSES).map(extract_sn_params, (sn_file for sn_file \
                                                   in sn_files))

/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:313: RuntimeWarning: overflow encountered in power
  cov_f = power(f*err/1.0857,2)
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:313: RuntimeWarning: overflow encountered in power
  cov_f = power(f*err/1.0857,2)
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:310: RuntimeWarning: overflow encountered in power
  f = power(10, -0.4*(mod - self.parent.data[band].filter.zp))
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:322: RuntimeWarning: invalid value encountered in multiply
  resids_list.append((f - self.parent.data[band].flux)*W)
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/kcorr.py:803: RuntimeWarning: invalid

Error: covariance matrix has negative diagonal element.
       Error for st not computed
Error: covariance matrix has negative diagonal element.
       Error for Tmax not computed
Error: covariance matrix has negative diagonal element.
       Error for Hmax not computed
Error: covariance matrix has negative diagonal element.
       Error for Jmax not computed
Error: covariance matrix has negative diagonal element.
       Error for gmax not computed
Error: covariance matrix has negative diagonal element.
       Error for rmax not computed
Error: covariance matrix has negative diagonal element.
       Error for st not computed
Error: covariance matrix has negative diagonal element.
       Error for Tmax not computed
Error: covariance matrix has negative diagonal element.
       Error for Hmax not computed
Error: covariance matrix has negative diagonal element.
       Error for Jmax not computed
Error: covariance matrix has negative diagonal element.
       Error for gmax not computed
Err

/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:310: RuntimeWarning: overflow encountered in power
  f = power(10, -0.4*(mod - self.parent.data[band].filter.zp))
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/model.py:322: RuntimeWarning: invalid value encountered in multiply
  resids_list.append((f - self.parent.data[band].flux)*W)
/home/tomas/anaconda3/envs/proposals/lib/python3.10/site-packages/snpy-2.5.3-py3.10-linux-x86_64.egg/snpy/mangle_spectrum.py:608: RuntimeWarning: overflow encountered in power
  resps[-1].append(num.power(10, -0.4*(mags[i][j] - fset[b].zp)))


In [5]:
params_df = pd.DataFrame(results)
params_df.to_csv('sn_parameters.csv', index=False)